# Project
## 3rd Notebook : Elasticsearch
##### Students:
* Lilia IZRI      (DS)
* Yacine MOKHTARI (DS)
* Alexandre COMBEAU (DS)

##### Report
[PENSER A METTRE UN LIEN ICI]


In [3]:
from elasticsearch import Elasticsearch
# from collections import Mapping


# database (es)
es = Elasticsearch("http://host.docker.internal:9200") # Same port when running elasticsearch with docker
# index
index = "test"

### Queries

#### 1. Using the Elasticsearch Python Client

#### Simple query to get 2 tweets just to check that it works as well 

In [5]:
query_body = {
    "match_all": {}
}
es.search(index=index, query=query_body, size=2)

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 783, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'test',
    '_type': '_doc',
    '_id': ' 297070331',
    '_score': 1.0,
    '_source': {'user': ' SincerelyHills',
     'tweet': ' RT @AlexJam91754067: Amber Heard says Johnny Depp\'s jealousy toward James Franco was "a nightmare." Here she is sneaking Franco into Johnny…',
     'date': ' Fri May 06 10:34:19 +0000 2022',
     'lat': ' 51.5384287',
     'lon': ' -0.0999051',
     'hashtags': ' ',
     'sentiment': ' 0',
     'id': ' 297070331'}},
   {'_index': 'test',
    '_type': '_doc',
    '_id': ' 1291493877053087746',
    '_score': 1.0,
    '_source': {'user': ' GT_LeighAsh1',
     'tweet': ' RT @HLNTV: "From the forehead to the nose, emotion flows." Body language expert points out where Johnny Depp\'s confusion &amp; disagreement com…',
     'date': ' Fri May 06 10:34:20 +0000 2022',
 

#### Query where we check for for a term with a boost

In [6]:
term2 = "lawyer"
query_with_boost = {
    "span_multi": {
      "match": {
        "prefix": { "tweet": { "value": term2, "boost": 1.08 } }
      }
    } 
}
es.search(index=index, query=query_with_boost, size=2)

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 8, 'relation': 'eq'},
  'max_score': 4.125748,
  'hits': [{'_index': 'test',
    '_type': '_doc',
    '_id': ' 3396680813',
    '_score': 4.125748,
    '_source': {'user': ' FerfeLaBat',
     'tweet': ' RT @TetrisphereJon6: Amber Heard just mentioned Kate Moss on the stand &amp; Johnny Depp’s lawyers celebrated. They just caught her in a lie ab…',
     'date': ' Fri May 06 12:38:17 +0000 2022',
     'lat': ' 24.9242995',
     'lon': ' -80.6278404',
     'hashtags': ' ',
     'sentiment': ' 1',
     'id': ' 3396680813'}},
   {'_index': 'test',
    '_type': '_doc',
    '_id': ' 1664452638',
    '_score': 4.125748,
    '_source': {'user': ' qierz_',
     'tweet': ' RT @TetrisphereJon6: Amber Heard just mentioned Kate Moss on the stand &amp; Johnny Depp’s lawyers celebrated. They just caught her in a lie ab…',
     'date': ' Fri May 06 12:38:18 +0000 2022',
  

#### Check for a term in the text of the tweet or the user 

In [7]:
term3 = "Kate"
query_body = {
    "multi_match": {
        "query": term3,
        "type": "most_fields",
        "fields": ["tweet", "user"]
    }    
}

es.search(index=index, query=query_body, size=2)

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 24, 'relation': 'eq'},
  'max_score': 4.4143505,
  'hits': [{'_index': 'test',
    '_type': '_doc',
    '_id': ' 2230823888',
    '_score': 4.4143505,
    '_source': {'user': ' 13SarahC',
     'tweet': ' RT @emma_lou_brown: So this is why Ben gave a fist bump when she mentioned Kate Moss. She has previously alleged JD pushed Kate down the st…',
     'date': ' Fri May 06 12:43:43 +0000 2022',
     'lat': ' 44.933143',
     'lon': ' 7.540121',
     'hashtags': ' ',
     'sentiment': ' -1',
     'id': ' 2230823888'}},
   {'_index': 'test',
    '_type': '_doc',
    '_id': ' 1392751646979940354',
    '_score': 4.0637655,
    '_source': {'user': ' nazmulmarketer1',
     'tweet': ' Case Update: Did Kate Moss Really Endure Johnny Depp\xa0Violence? https://t.co/3ByEM7fSaS',
     'date': ' Fri May 06 12:53:08 +0000 2022',
     'lat': ' 24.4769288',
     'lon': ' 90.

#### Query with a regexp and highlight

In [8]:
field = "user"
regexp = "no.*"

body_query = {
  "query": {
    "regexp": {
      field: {
        "value": regexp,
        "flags": "ALL",
        "case_insensitive": False,
        "max_determinized_states": 10000,
        "rewrite": "constant_score"
      }
    }
  },
  "highlight" : {    
        "pre_tags" : ["<em>"],
        "post_tags" : ["</em>"],
        "fields" : {
            "user" : {}
        }
   }
}
es.search(index=index, body=body_query, size=2)


/tmp/ipykernel_7835/3593569734.py:24: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.search(index=index, body=body_query, size=2)


{'took': 6,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 7, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'test',
    '_type': '_doc',
    '_id': ' 2407110908',
    '_score': 1.0,
    '_source': {'user': ' NooraBK',
     'tweet': ' RT @NaserMestarihi: She’s pretending to be scared as Johnny Depp’s leaving the courtroom. This bitch is crazy. 😂 https://t.co/DVRb6ZvhI3',
     'date': ' Fri May 06 12:42:35 +0000 2022',
     'lat': ' 25.2653471',
     'lon': ' 55.2924914',
     'hashtags': ' ',
     'sentiment': ' -1',
     'id': ' 2407110908'},
    'highlight': {'user': ['<em>NooraBK</em>']}},
   {'_index': 'test',
    '_type': '_doc',
    '_id': ' 2164802278',
    '_score': 1.0,
    '_source': {'user': ' noora1326',
     'tweet': " RT @paulkart76: I'm thankful Johnny Depp is even still alive at all. Amber Heard is one of the scariest women I've ever seen. This is her i…",
     'date': ' Fri May 06 12

#### 2. Using the extended elasticsearch_dsl API (especially, the search engine in it)

Basically, this is just a "simpler" way to express the previous 

In [11]:
from elasticsearch_dsl import Search

# Search engine
s = Search(using=es, index=index)

In [12]:
# Query where "heard" appears in the tweet text

term = "heard"
result = s.query('match', tweet=term).execute()
for hit in result:
    print(hit.tweet)

 RT @Familoo: I’m not going to retweet any of the footage of the Heard v Depp (Depp v Heard?) trial - but haven’t the American courts heard…
 Viikon tauko #depp #heard oikeudenkäynnistä tulee tarpeeseen?
 RT @mariomojc: Cuando Bredehoft pregunta sobre las peleas de Heard con Depp sobre James Franco, Heard testifica: "Fue una pesadilla". Antes…
 @Glenn__Kenny Are you watching the Depp Heard case?
 @StanleyCohenLaw Depp says his career was ruined by Heard....Heard says her accusations of abuse are valid, Depp says he was abused too...
 RT @SystemicDunking: I feel like I'm watching Amber heard playing the role of Amber heard in a made for tv movie about Johnny Depp. Its wil…
 Amber Heard x Johnny Depp les deux sont complètement toxiques
 RT @ellisgreg: Believe all women? Even Amber Heard? https://t.co/7OCYTx6qqV
 RT @ellisgreg: Believe all women? Even Amber Heard? https://t.co/7OCYTx6qqV
 RT @ellisgreg: Believe all women? Even Amber Heard? https://t.co/7OCYTx6qqV
